In [1]:
# This model predicts alphabed(A-J) given the image using Neural Network with introduced non linearities using RELU

# Import Dependencies
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
#Pre processed pickle file
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    # Loading pickle file to memory which is a dictionary containing training data and labels
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
# Size of each image 28*28 pixels
image_size = 28
# Data contains alphabets from A to J(10)
num_labels = 10

# Flattening all the data to 2-d matrix
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# Formula to calculate accuracy of our model
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [27]:
# Running Stochastic Gradient Descent as optimizer with batch size of 128
batch_size = 128
# Defining custom graph for our tensorflow model
graph = tf.Graph()

with graph.as_default():
    # Defining training data and labels as placeholder as they will change in each iteration
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # Defining validation and test data as constants which are stored in memory
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Model is Input->(Logistic Regression)->(1024 RELUs)->(Logistic Regression)->Output
    
    # Defining weights and biases as variables with respective initialization
    
    # Weights and biases for the first regressor
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
    biases1 = tf.Variable(tf.zeros([1024]))
    
    # Weights and biases for the seconf regressor
    weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Output of first regressor, Y = W*X + B
    out_lc = tf.matmul(tf_train_dataset, weights1) + biases1
    
    # Above output is fed into RELUs to introduce non-linearity
    out_relu = tf.nn.relu(out_lc)
    
    # Logits are output of the second regressor
    logits = tf.matmul(out_relu, weights2) + biases2
    
    # Finding Loss by comparing softmax probability distribution of logits and one hot encoding of training labels
    # Using L2 Regulizer to prevent model from over-fitting by penalizing weights
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 0.001*tf.nn.l2_loss(weights1) + 0.001*tf.nn.l2_loss(weights2)) 
    
    # Gradient Descent Optimizer with step size = 0.05
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Calculating predictions for training, validation and testing data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
    test_prediction =  tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [32]:
# Iterating for 3000 times on the above model
num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval({keep_prob: 1.0}), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval({keep_prob: 1.0}), test_labels))

Minibatch loss at step 0: 506.587830
Minibatch accuracy: 10.9%
Validation accuracy: 34.8%
Minibatch loss at step 500: 8.665834
Minibatch accuracy: 77.3%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 17.562141
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 1500: 14.988734
Minibatch accuracy: 68.0%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 16.914795
Minibatch accuracy: 72.7%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 3.931273
Minibatch accuracy: 75.0%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 13.519794
Minibatch accuracy: 77.3%
Validation accuracy: 79.2%
Minibatch loss at step 3500: 5.476126
Minibatch accuracy: 78.1%
Validation accuracy: 79.5%
Minibatch loss at step 4000: 7.192588
Minibatch accuracy: 73.4%
Validation accuracy: 81.1%
Minibatch loss at step 4500: 4.782463
Minibatch accuracy: 70.3%
Validation accuracy: 81.0%
Minibatch loss at step 5000: 8.426310
Minibatch accuracy: 73.4%
Validation accuracy: 81.